# Sample Notebook to deploy a HF MLFlow model to an AML workspace using Python SDK

### Define required parameters

Update these parameters to test deployments in your own workspace

In [ ]:
# Define Parameters
subscription_id="" # Replace with your subscription ID
resource_group="" # Replace with your resource group name
workspace_name="" # Replace with your workspace name
registry_name="" # Replace with your registry name
endpoint_name="" # Replace with your endpoint name
deployment_name="" # Replace with your deployment name
model_name="" # Name of the model to be deployed
sku_name="Standard_DS2_v2" # Name of the sku(instance type) Check the model-list(can be found in the parent folder(inference)) to get the most optimal sku for your model (Default: Standard_DS2_v2)

### Install required libraries

In [ ]:
# Skip this cell if all libraries are already installed
# The required libraries can be installed in the local environment using the following command:
%pip install azure-ai-ml==1.2.0 azure-identity

### Import required libraries

In [ ]:
import os
import json
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Model, ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.identity import DefaultAzureCredential

### Setup the ML Clients

In [ ]:
# Create an ML Client to interact with your workspace
ws_client = MLClient(DefaultAzureCredential(), subscription_id=subscription_id, resource_group_name=resource_group, workspace_name=workspace_name)

# Create an ML Client to interact with the registry
reg_client = MLClient(DefaultAzureCredential(), subscription_id=subscription_id, resource_group_name=resource_group, registry_name=registry_name)

### Validate the model in the registry

In [ ]:
# This cell validates the existence of the model in the registry
try:
    model_list = reg_client.models.list(name=model_name)
    version=list(model_list)[0].version
    model = reg_client.models.get(model_name, version)
    print(f"Model validated successfully. Using model {model_name}, version {version}")
except:
    print("Model not found in the registry. Check the registry for the list of supported models")

### Create an online endpoint

In [ ]:
# Check if the endpoint already exists in the workspace
try:
    endpoint = ws_client.online_endpoints.get(endpoint_name)
    print("---Endpoint already exists---")
except:
    # Create an online endpoint if it doesn't exist

    # Define the endpoint
    endpoint = ManagedOnlineEndpoint(name=endpoint_name, description="Test endpoint for model")

    # Trigger the endpoint creation
    try:
        ws_client.begin_create_or_update(endpoint).wait()
        print("\n---Endpoint created successfully---\n")
    except Exception as err:
        raise RuntimeError(f"Endpoint creation failed. Detailed Response:\n{err}") from err

### Create an online endpoint-deployment

In [ ]:
# Define the deployment
# Update the model version as necessary
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model.id,
    instance_type=sku_name,
    instance_count=1,
)

# Trigger the deployment creation
try:
    ws_client.begin_create_or_update(deployment).wait()
    print("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(f"Deployment creation failed. Detailed Response:\n{err}") from err

### Inference Testing

In [ ]:
# For testing the inference of the deployment, create a *sample-request.json* file in the same folder as the notebook. 
# Sample inputs for different tasks can be found in the *sample-inputs* folder under *inference*.
# You can update the file with your own custom input and test the deployment.
# Note: For the fill-mask task, ensure that the correct mask token is used in the sample request.

# Invoke the deployment using the given input
try:
    with open("./sample-request.json", "r") as f:
        print(f"Input: \n{f.read()}")
    output = ws_client.online_endpoints.invoke(
        endpoint_name=endpoint_name,
        deployment_name=deployment_name,
        request_file="./sample-request.json",
    )
    print(f"Output: \n{output}\n")
except Exception as err:
    raise RuntimeError(f"Inference to endpoint: {endpoint_name} and deployment: {deployment_name} failed. Detailed Response:\n{err}") from err

### Delete all resources (Optional)

In [ ]:
# Run this cell to delete all resources created as part of this notebook
try:
    ws_client.online_deployments.begin_delete(name=deployment_name, endpoint_name=endpoint_name)
    print("\n---Deployment deletion triggered successfully---\n")
except Exception as err:
    raise RuntimeError(f"Deployment {deployment_name} deletion failed. Detailed Response:\n{err}") from err